In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="2"

Thu Aug 24 17:34:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   24C    P8    13W / 230W |      6MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [3]:
n_mels = 1205
n_outputs = 61
n_frames = 400

### Model

In [4]:
from model.model_glowtts_sid import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, enc_hidden_dim=256, out_dim=61, dec_hidden_dim=256)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
print('done')

done


### Load

In [5]:
save_dir = '/data/scpark/save/lips/train08.21-1/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if False:
    step, model, _, optimizer = load(save_dir, 50421, model, None, optimizer)

total 646940
-rw-rw-r-- 1 scpark scpark        40  8월 24 17:34 events.out.tfevents.1692865738.GPUSVR01
-rw-rw-r-- 1 scpark scpark 662455517  8월 24 10:46 save_130000


### Dataset

In [6]:
import os
from data.arkit_dataset import LipsDataset, CombinedDataset, CombinedCollate

root_dir = '/data/speech/digital_human/preprocessed/'

def get_files(dir):
    data = []
    files = sorted([os.path.join(dir, file) for file in os.listdir(dir)])
    for file in files:
        if file.endswith('.npy') and 'ARKit' in file:
            data.append(file)
        if os.path.isdir(file):
            data.extend(get_files(os.path.join(dir, file)))
    return data

files = get_files(root_dir)
print(len(files))

train_datasets = []
test_datasets = []

def get_sid(file):
    names = ['No Speaker', 'jeewonPark', 'jinwooOh', 'kyuchulLee', 'kyuseokKim', 'nohsikPark', 'soochulPark', 'yehunHwang']
    for sid, name in enumerate(names):
        if name in file:
            return sid
    return 0

for file in files:
    sid = get_sid(file)
    print(sid, file)
    dataset = LipsDataset(file, n_mels, n_frames, sid=sid, mel=False)
    if '_10_' in file:
        test_datasets.append(dataset)
    else:
        train_datasets.append(dataset)
print(len(train_datasets), len(test_datasets))

39
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_1_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_2_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_3_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_4_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_5_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_6_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_7_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_8_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_001_9_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_002_10_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_002_1_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_002_2_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_002_3_iPhone_raw.npy
0 /data/speech/digital_human/preprocessed/MH_ARKit_002_4_iPh

In [7]:
train_loader = torch.utils.data.DataLoader(CombinedDataset(train_datasets), 
                                           num_workers=16, shuffle=True, batch_size=32, collate_fn=CombinedCollate())
test_loader = torch.utils.data.DataLoader(CombinedDataset(test_datasets), 
                                          num_workers=10, shuffle=True, batch_size=10, collate_fn=CombinedCollate())
print('done')

done


In [8]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
wav2vec = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to(device)
print('done')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


done


In [9]:
from torchaudio.transforms import Resample
resample = Resample(24000, 16000).to(device)

def get_probs(wav, size):
    # 24k to 16k
    wav = resample(wav)
    # Get probs.
    with torch.no_grad():
        logits = wav2vec(wav).logits
        probs = torch.softmax(logits, dim=-1)
    probs = probs.transpose(1, 2)
    print(probs.shape)
    # Resizing
    probs = F.interpolate(probs, size=size, mode='linear').detach()
    print(probs.shape)
    return probs


In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        targets = torch.Tensor(batch['blend']).transpose(1, 2).to(device)
        inputs = get_probs(torch.Tensor(batch['wav']).to(device), targets.shape[-1])
        sid = torch.Tensor(batch['sid']).int().to(device)
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets, sid)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 1000 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 1000 == 0:
            display.clear_output()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                targets = torch.Tensor(batch['blend']).transpose(1, 2).to(device)
                inputs = get_probs(torch.Tensor(batch['wav']).to(device), targets.shape[-1])
                sid = torch.Tensor(batch['sid']).int().to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets, sid)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)
            
            plt.figure(figsize=[18, 4])
            librosa.display.specshow(targets[0].data.cpu().numpy(), cmap='magma')
            plt.show()

            model.eval()
            with torch.no_grad():
                y_pred = model.inference(inputs, sid)

            plt.figure(figsize=[18, 4])
            librosa.display.specshow(y_pred[0].data.cpu().numpy(), cmap='magma')
            plt.show()
            
            for i in [20, 37]:
                plt.figure(figsize=[18, 2])
                plt.title(str(i))
                plt.plot(targets[0].data.cpu().numpy()[i])
                plt.plot(y_pred[0].data.cpu().numpy()[i])
                plt.show()

        if step % 10000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

torch.Size([32, 1205, 666])
torch.Size([32, 1205, 400])
50421
loss -5.039287567138672
50421 -5.039287567138672
torch.Size([32, 1205, 666])
torch.Size([32, 1205, 400])
50422
loss -5.04115104675293
50422 -5.04115104675293
torch.Size([32, 1205, 666])
torch.Size([32, 1205, 400])
50423
loss -5.070566177368164
50423 -5.070566177368164
torch.Size([32, 1205, 666])
torch.Size([32, 1205, 400])
50424
loss -5.140014171600342
50424 -5.140014171600342
torch.Size([32, 1205, 666])
torch.Size([32, 1205, 400])
50425
loss -5.125100612640381
50425 -5.125100612640381
torch.Size([32, 1205, 666])
torch.Size([32, 1205, 400])
50426
loss -5.0268168449401855
50426 -5.0268168449401855
torch.Size([32, 1205, 666])
torch.Size([32, 1205, 400])
50427
loss -5.094602108001709
50427 -5.094602108001709
torch.Size([32, 1205, 666])
torch.Size([32, 1205, 400])


In [ ]:
save(save_dir, step, model, None, optimizer)

In [ ]:
print('done')